In [2]:
import numpy as np
import pandas as pd 
import traceback
import yfinance as yf

# Helper class: Fetch Stock Data 

To Do List: 
* Get data within a specific range

In [24]:
class StockFetcher: 
    def __fetch_historical_data(self, ticker, time_unit, period=None): 
        try:
            if not period: 
                if time_unit in ['ytd', 'max']: 
                    period_str = f'{time_unit}'
                else:
                    raise ValueError('Period must be defined for a given time unit')
            else: 
                period_str = f'{period}{time_unit}'

            data = yf.Ticker(ticker).history(period=period_str)
            return data 
        except Exception as e: 
            print(f'Validation error: {e}')
            traceback.print_exc()
            return None 

    def get_stock_data(self, ticker, time_unit, period=None): 
        if time_unit not in ['d', 'mo', 'y', 'ytd', 'max']: 
            raise ValueError('Time unit not valid') # Later be changed to a HTTP error 
        
        match time_unit: 
            case 'd': 
                if period not in [1, 5]: 
                    raise ValueError('Period not valid') # Later be changed to a HTTP error 
            case 'mo': 
                if period not in [1, 3, 6]: 
                    raise ValueError('Period not valid') # Later be changed to a HTTP error 
            case 'y': 
                if period not in [1, 2, 5, 10]: 
                    raise ValueError('Period not valid') # Later be changed to a HTTP error 
            case 'ytd' | 'max': 
                period = None
            case _: 
                raise ValueError('Time unit not valid')
        
        try: 
            data = self.__fetch_historical_data(ticker, time_unit, period)
            return data
        except Exception as e: 
            print(f'Validation error: {e}')
            traceback.print_exc()
            return None

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.098943,0.099373,0.098943,0.098943,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094211,0.094211,0.093781,0.093781,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.087328,0.087328,0.086898,0.086898,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089049,0.089479,0.089049,0.089049,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.091630,0.092061,0.091630,0.091630,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-23 00:00:00-04:00,227.339996,229.449997,225.809998,226.470001,54146000,0.0,0.0
2024-09-24 00:00:00-04:00,228.649994,229.350006,225.729996,227.369995,43556100,0.0,0.0
2024-09-25 00:00:00-04:00,224.929993,227.289993,224.020004,226.369995,42308700,0.0,0.0


In [17]:
stock_fetcher = StockFetcher() 
stock = stock_fetcher.get_stock_data(ticker='AAPL', time_unit='max')  

AAPL: Period 'maxnone' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']


# Helper Class: Performs Regression Analysis 

To Do List: 
* Regression analysis within a specific range 

In [48]:
class LinearRegresion: 

    def __get_coefficients(self, asset, market):
        avg_market_return = market['Returns'].mean()
        avg_asset_return = asset['Returns'].mean()

        covariance = market['Returns'].cov(asset['Returns'])
        variance = market['Returns'].var() 

        beta = covariance / variance 
        alpha = avg_asset_return - beta * avg_market_return

        expected_returns = alpha + beta*avg_market_return
        
        return alpha, beta, expected_returns

    def linear_regression(self, ticker): 
        stock_fetcher = StockFetcher() 
        asset = stock_fetcher.get_stock_data(ticker=ticker, time_unit='y', period=10)    
        market = stock_fetcher.get_stock_data(ticker='SPY', time_unit='y', period=10)    

        date_match = pd.merge(asset, market, left_index=True, right_index=True, how='inner').index 

        asset_filtered = asset.loc[date_match]
        market_filtered = market.loc[date_match]

        asset_filtered['Returns'] = asset_filtered.Close.pct_change()
        market_filtered['Returns'] = market_filtered.Close.pct_change()

        alpha, beta, expected_returns = self.__get_coefficients(asset_filtered, market_filtered)

        return alpha, beta, expected_returns

lg = LinearRegresion() 

print(lg.linear_regression('AAPL'))
print(lg.linear_regression('AMZN'))
print(lg.linear_regression('NVDA'))

(0.0004157690295614881, 1.2098946610699466, 0.0010865688851274568)
(0.0005492181669710213, 1.1548074964868835, 0.0011894761388219406)
(0.0017250252599740187, 1.7478948558670373, 0.0026941076558110594)
